In [1]:
import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), 'utils'))
from configuration import *
import torch
import pprint
import pNN
from utils import *

args = parser.parse_args([])
args.e_train = 0.05
args.N_train = 10
args.SEED = 3
args = FormulateArgs(args)

In [2]:
print(f'Training network on device: {args.DEVICE}.')
MakeFolder(args)

train_loader, datainfo = GetDataLoader(args, 'train')
valid_loader, datainfo = GetDataLoader(args, 'valid')
test_loader, datainfo = GetDataLoader(args, 'test')
pprint.pprint(datainfo)

SetSeed(args.SEED)

setup = f"data_{args.DATASET:02d}_{datainfo['dataname']}_seed:{args.SEED:02d}_epsilon:{args.e_train}.model"
print(f'Training setup: {setup}.')

msglogger = GetMessageLogger(args, setup)
msglogger.info(f'Training network on device: {args.DEVICE}.')
msglogger.info(f'Training setup: {setup}.')
msglogger.info(datainfo)

topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
msglogger.info(f'Topology of the network: {topology}.')

pnn = pNN.pNN(topology, args).to(args.DEVICE)

lossfunction = pNN.pNNLoss(args).to(args.DEVICE)
optimizer = torch.optim.Adam(pnn.GetParam(), lr=args.LR)

if args.PROGRESSIVE:
    pnn, best = train_pnn_progressive(pnn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)
else:
    pnn, best = train_pnn(pnn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)

if best:
    if not os.path.exists(f'{args.savepath}/'):
        os.makedirs(f'{args.savepath}/')
    torch.save(pnn, f'{args.savepath}/{setup}')
    msglogger.info('Training if finished.')
else:
    msglogger.warning('Time out, further training is necessary.')

Training network on device: cpu.
{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}
Training setup: data:acuteinflammation_seed:03_epsilon:0.05.model.
torch.Size([10, 70, 8])
torch.Size([8])
torch.Size([10, 70, 7])
torch.Size([7])
torch.Size([10, 70, 8])
torch.Size([8])
torch.Size([10, 70, 7])
torch.Size([7])
torch.Size([10, 23, 8])
torch.Size([8])
torch.Size([10, 23, 7])
torch.Size([7])
torch.Size([10, 23, 8])
torch.Size([8])
torch.Size([10, 23, 7])
torch.Size([7])
| Epoch:      0 | Train loss: 6.9308e-01 | Valid loss: 6.9054e-01 | Train acc: 0.5114 ± 0.0262 | Valid acc: 0.6087 ± 0.0000 | patience:   0 | lr: 0.1 | Epoch time: 0.5 |
torch.Size([10, 70, 8])
torch.Size([8])
torch.Size([10, 70, 7])
torch.Size([7])
torch.Size([10, 70, 8])
torch.Size([8])
torch.Size([10, 70, 7])
torch.Size([7])
torch.Size([10, 23, 8])
torch.Size([8])
torch.Size([10, 23, 7])
torch.Size([7])
torch.Size([10, 23, 8])
torch.Size([8])
torch.Size([10, 2

KeyboardInterrupt: 

In [3]:
evaluator = Evaluator(args)

In [4]:
for x_train, y_train in test_loader:
    train_result = evaluator(pnn, x_train, y_train)
    train_acc, train_std, train_power, train_area = train_result['acc'], train_result['std'], train_result['power'], train_result['area']
train_acc

torch.Size([10, 25, 8])
torch.Size([8])
torch.Size([10, 25, 7])
torch.Size([7])


1.0

In [5]:
pnn.UpdateVariation(100, 0.2)
for x_train, y_train in test_loader:
    train_result = evaluator(pnn, x_train, y_train)
    train_acc, train_std, train_power, train_area = train_result['acc'], train_result['std'], train_result['power'], train_result['area']
train_acc

torch.Size([100, 25, 8])
torch.Size([8])
torch.Size([100, 25, 7])
torch.Size([7])


0.9076000000000001

In [18]:
pnn.RemoveFault()
pnn.SampleFault(0)
for x_train, y_train in test_loader:
    train_result = evaluator(pnn, x_train, y_train)
    train_acc, train_std, train_power, train_area = train_result['acc'], train_result['std'], train_result['power'], train_result['area']
train_acc

torch.Size([100, 25, 8])
torch.Size([8])
torch.Size([100, 25, 7])
torch.Size([7])


0.9228000000000001